# Sanity Checks for Climate Pipeline Tables

This notebook performs simple sanity checks on the main DuckDB tables:

- `bronze_daily_weather`
- `silver_daily_weather_features`
- `silver_monthly_climate`
- `climatology_city_month`
- `gold_city_month_anomalies`
- `gold_city_anomaly_events`
- `gold_city_anomaly_lags`
- `gold_ml_features`

Use this notebook whenever you modify ingestion or dbt models to confirm that
row counts, date ranges, and basic distributions still look reasonable.

In [1]:
import duckdb
import pandas as pd

DB_PATH = "../data/warehouse/climate.duckdb"
con = duckdb.connect(DB_PATH, read_only=True)

## Helper functions

In [2]:
def table_exists(name: str) -> bool:
    try:
        con.execute(f"SELECT 1 FROM {name} LIMIT 1")
        return True
    except Exception:
        return False

def describe_table(name: str, sample_rows: int = 5):
    print(f"\n=== {name} ===")
    if not table_exists(name):
        print("Table does not exist.")
        return
    
    row_count = con.execute(f"SELECT COUNT(*) AS n FROM {name}").fetchone()[0]
    print(f"Row count: {row_count}")
    
    df_head = con.execute(f"SELECT * FROM {name} LIMIT {sample_rows}").df()
    print("\nSample rows:")
    display(df_head)

## 1. Bronze layer

In [3]:
describe_table('bronze_daily_weather')


=== bronze_daily_weather ===
Row count: 65748

Sample rows:


,city_id,city_name,country_code,city_slug,year,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,dew_point_2m_mean,precipitation_sum,wind_speed_10m_max,shortwave_radiation_sum
0,3,Lisbon,PT,lisbon,2010,2010-12-31,14.4,12.6,13.3,11.9,9.7,25.3,6.03
1,1,Los Angeles,US,los_angeles,1993,1993-12-31,21.6,4.9,12.2,-2.3,0.0,10.5,12.43
2,1,Los Angeles,US,los_angeles,2009,2009-12-31,19.5,5.2,11.2,3.7,0.0,8.9,11.80
3,1,Los Angeles,US,los_angeles,2023,2023-12-31,15.4,8.7,11.9,8.8,0.3,15.5,5.72
4,4,Porto,PT,porto,1992,1992-12-31,10.1,3.6,7.0,3.6,0.0,9.8,4.82


In [4]:
if table_exists("bronze_daily_weather"):
    df_bronze = con.execute("""
        SELECT city_id, MIN(date) AS min_date, MAX(date) AS max_date, COUNT(*) AS n
        FROM bronze_daily_weather
        GROUP BY city_id
        ORDER BY city_id
    """).df()
    df_bronze

## 2. Silver layer

In [5]:
describe_table('silver_daily_weather_features')


=== silver_daily_weather_features ===
Row count: 65748

Sample rows:


,city_id,city_name,country_code,date,tmax_c,tmin_c,tmean_c,dewpoint_c,precip_mm,wind_max_ms,sw_radiation,year,month,day_of_year,is_summer,is_heat_day,is_tropical_night,is_heavy_precip_day
0,3,Lisbon,PT,2010-12-31,14.4,12.6,13.3,11.9,9.7,25.3,6.03,2010,12,365,0,0,0,0
1,1,Los Angeles,US,1993-12-31,21.6,4.9,12.2,-2.3,0.0,10.5,12.43,1993,12,365,0,0,0,0
2,1,Los Angeles,US,2009-12-31,19.5,5.2,11.2,3.7,0.0,8.9,11.80,2009,12,365,0,0,0,0
3,1,Los Angeles,US,2023-12-31,15.4,8.7,11.9,8.8,0.3,15.5,5.72,2023,12,365,0,0,0,0
4,4,Porto,PT,1992-12-31,10.1,3.6,7.0,3.6,0.0,9.8,4.82,1992,12,366,0,0,0,0


In [6]:
if table_exists("silver_daily_weather_features"):
    df_silver_daily = con.execute("""
        SELECT
            city_id,
            MIN(date) AS min_date,
            MAX(date) AS max_date,
            COUNT(*) AS n
        FROM silver_daily_weather_features
        GROUP BY city_id
        ORDER BY city_id
    """).df()
    df_silver_daily

In [7]:
describe_table('silver_monthly_climate')


=== silver_monthly_climate ===
Row count: 2160

Sample rows:


,city_id,city_name,country_code,year,month,days_in_month,avg_tmax_c,avg_tmin_c,avg_tmean_c,avg_dewpoint_c,avg_wind_max_ms,avg_sw_radiation,total_precip_mm,heat_day_count,tropical_night_count,heavy_precip_day_count,summer_day_count
0,4,Porto,PT,1992,12,31,13.748387,7.980645,10.574194,7.290323,19.035484,5.952903,151.4,0.0,0.0,4.0,0.0
1,3,Lisbon,PT,2010,11,30,16.166667,12.253333,14.040000,10.130000,23.930000,9.005333,81.9,0.0,0.0,4.0,0.0
2,4,Porto,PT,1992,1,31,12.041935,3.354839,6.916129,3.116129,14.806452,8.530323,54.0,0.0,0.0,1.0,0.0
3,1,Los Angeles,US,2016,11,30,25.123333,11.353333,17.286667,3.536667,10.616667,12.982667,18.8,8.0,0.0,1.0,0.0
4,1,Los Angeles,US,1982,10,31,27.629032,13.077419,19.725806,7.138710,12.945161,16.865161,5.7,11.0,0.0,0.0,0.0


In [8]:
if table_exists("silver_monthly_climate"):
    df_silver_monthly = con.execute("""
        SELECT
            city_id,
            MIN(year) AS min_year,
            MAX(year) AS max_year,
            COUNT(*) AS n
        FROM silver_monthly_climate
        GROUP BY city_id
        ORDER BY city_id
    """).df()
    df_silver_monthly

## 3. Gold climatology & anomalies

In [9]:
describe_table('climatology_city_month')


=== climatology_city_month ===
Row count: 48

Sample rows:


,city_id,city_name,country_code,month,climatology_tmean_c,climatology_tmax_c,climatology_tmin_c,climatology_total_precip_mm,climatology_tmean_std_c,climatology_total_precip_std_mm
0,3,Lisbon,PT,11,14.635556,16.981667,12.628667,84.806667,1.095566,59.508654
1,2,San Francisco,US,10,15.336129,20.530323,11.542581,28.820000,0.796351,23.413589
2,2,San Francisco,US,6,15.888333,21.028333,11.706111,3.260000,0.807698,4.837540
3,1,Los Angeles,US,7,24.414731,31.476344,18.048280,0.653333,1.477673,2.016267
4,1,Los Angeles,US,4,16.138222,22.933333,9.941556,23.363333,1.735023,26.709129


In [10]:
if table_exists("climatology_city_month"):
    df_clim = con.execute("""
        SELECT
            city_id,
            month,
            AVG(climatology_tmean_c) AS avg_clim_tmean
        FROM climatology_city_month
        GROUP BY city_id, month
        ORDER BY city_id, month
    """).df()
    df_clim.head(24)

In [11]:
describe_table('gold_city_month_anomalies')


=== gold_city_month_anomalies ===
Row count: 2160

Sample rows:


,city_id,city_name,country_code,year,month,month_name,season,avg_tmean_c,avg_tmax_c,avg_tmin_c,...,anomaly_tmean_c,anomaly_tmax_c,anomaly_tmin_c,anomaly_total_precip_mm,zscore_tmean,zscore_total_precip,is_positive_temp_anomaly,is_negative_temp_anomaly,is_strong_positive_temp_anomaly,is_strong_negative_temp_anomaly
0,3,Lisbon,PT,1980,1,January,DJF,11.454839,13.448387,9.667742,...,0.278387,-0.039570,0.579355,-29.923333,0.298337,-0.470337,False,False,False,False
1,3,Lisbon,PT,1980,2,February,DJF,11.989655,14.375862,9.886207,...,0.253140,0.114828,0.360965,10.553333,0.251350,0.223452,False,False,False,False
2,3,Lisbon,PT,1980,3,March,MAM,12.932258,15.177419,10.983871,...,-0.594946,-1.435054,0.047097,-4.266667,-0.610346,-0.118011,False,False,False,False
3,3,Lisbon,PT,1980,4,April,MAM,15.676667,19.213333,12.623333,...,1.018111,1.601000,0.491667,-14.143333,0.946821,-0.432949,True,False,False,False
4,3,Lisbon,PT,1980,5,May,MAM,16.051613,18.748387,13.761290,...,-0.678710,-1.164946,-0.306237,14.026667,-0.637073,0.456573,False,False,False,False


In [12]:
if table_exists("gold_city_month_anomalies"):
    df_anom = con.execute("""
        SELECT
            city_id,
            MIN(year) AS min_year,
            MAX(year) AS max_year,
            COUNT(*) AS n
        FROM gold_city_month_anomalies
        GROUP BY city_id
        ORDER BY city_id
    """).df()
    df_anom

In [13]:
if table_exists("gold_city_month_anomalies"):
    df_anom_stats = con.execute("""
        SELECT
            city_id,
            AVG(anomaly_tmean_c) AS mean_anom,
            STDDEV(anomaly_tmean_c) AS std_anom
        FROM gold_city_month_anomalies
        GROUP BY city_id
        ORDER BY city_id
    """).df()
    df_anom_stats

## 4. Anomaly events and lags

In [14]:
describe_table('gold_city_anomaly_events')


=== gold_city_anomaly_events ===
Row count: 2160

Sample rows:


,city_id,city_name,year,month,temperature_anomaly,z_score,is_hot_event,is_cold_event,is_extreme_event
0,3,Lisbon,1980,1,0.278387,0.298337,False,False,False
1,3,Lisbon,1980,2,0.253140,0.251350,False,False,False
2,3,Lisbon,1980,3,-0.594946,-0.610346,False,False,False
3,3,Lisbon,1980,4,1.018111,0.946821,False,False,False
4,3,Lisbon,1980,5,-0.678710,-0.637073,False,False,False


In [15]:
if table_exists("gold_city_anomaly_events"):
    df_events = con.execute("""
        SELECT
            city_id,
            SUM(is_hot_event::INT) AS n_hot,
            SUM(is_cold_event::INT) AS n_cold,
            SUM(is_extreme_event::INT) AS n_extreme,
            COUNT(*) AS n_total
        FROM gold_city_anomaly_events
        GROUP BY city_id
        ORDER BY city_id
    """).df()
    df_events

In [16]:
describe_table('gold_city_anomaly_lags')


=== gold_city_anomaly_lags ===
Row count: 156

Sample rows:


,city_id,city_name,other_city_id,other_city_name,lag_months,n_observations,anomaly_correlation
0,3,Lisbon,1,Los Angeles,-6,534,-0.022402
1,3,Lisbon,1,Los Angeles,-5,535,0.012306
2,3,Lisbon,1,Los Angeles,-4,536,0.036988
3,3,Lisbon,1,Los Angeles,-3,537,0.039948
4,3,Lisbon,1,Los Angeles,-2,538,-0.036506


In [17]:
if table_exists("gold_city_anomaly_lags"):
    df_lags = con.execute("""
        SELECT
            city_id,
            other_city_id,
            MIN(lag_months) AS min_lag,
            MAX(lag_months) AS max_lag,
            MAX(ABS(anomaly_correlation)) AS max_abs_corr
        FROM gold_city_anomaly_lags
        GROUP BY city_id, other_city_id
        ORDER BY city_id, other_city_id
    """).df()
    df_lags

## 5. ML features table

In [18]:
describe_table('gold_ml_features')


=== gold_ml_features ===
Row count: 2160

Sample rows:


,city_id,city_name,year,month,sin_month,cos_month,anomaly_tmean_c,roll_mean_3,roll_mean_6,roll_std_3,roll_std_6,delta_1m,delta_3m,max_lagged_corr,lead_lag_months,is_event_next_month
0,4,Porto,1980,1,0.500000,8.660254e-01,0.377312,0.377312,0.377312,NaN,NaN,NaN,NaN,0.878895,0,0
1,4,Porto,1980,2,0.866025,5.000000e-01,0.351872,0.364592,0.364592,0.017989,0.017989,-0.025440,NaN,0.878895,0,0
2,4,Porto,1980,3,1.000000,6.123234e-17,-0.958925,-0.076580,-0.076580,0.764239,0.764239,-1.310797,NaN,0.878895,0,0
3,4,Porto,1980,4,0.866025,-5.000000e-01,0.565778,-0.013758,0.084009,0.825496,0.701805,1.524703,0.188466,0.878895,0,0
4,4,Porto,1980,5,0.500000,-8.660254e-01,-1.200538,-0.531228,-0.172900,0.957685,0.836307,-1.766315,-1.552410,0.878895,0,0


In [19]:
if table_exists("gold_ml_features"):
    df_ml = con.execute("""
        SELECT
            city_id,
            COUNT(*) AS n,
            AVG(is_event_next_month) AS event_rate
        FROM gold_ml_features
        GROUP BY city_id
        ORDER BY city_id
    """).df()
    df_ml

## Notes

If any of these checks start to look suspicious (e.g. zero rows, missing years, strange anomaly mean/std),
it’s a strong signal to:

- Re-run the relevant dbt models (bronze → silver → gold)
- Check path assumptions in the bronze model (file locations)
- Confirm that ingestion completed without errors
- Revisit the transformations in the dbt SQL

This notebook is meant to be quick to run and quick to scan, so you can use it as a regression
check whenever you change the pipeline.